In [1]:
!pip install -q underthesea
!pip install -q rouge-score
!pip install -q nltk
!pip install -q flash_attn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 53.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.0 MB/s eta 0:00:00


In [2]:
import os, math, random
import json
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint
from torch.nn import TransformerDecoderLayer, TransformerDecoder, MultiheadAttention, LayerNorm
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    CLIPImageProcessor,
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW
from underthesea import word_tokenize
import easyocr
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import torchvision.transforms as transforms
from typing import Optional
import nltk
nltk.download('punkt')

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

2025-05-19 07:50:44.527304: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747641044.753680      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747641044.814875      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
device      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
IMG_SIZE    = 448
PATCH_LEN   = (IMG_SIZE // 8) ** 2 + 1      # 197
MAX_LEN     = 15                             # max text length
MEM_LEN     = PATCH_LEN + MAX_LEN           # 229
BEAM_SIZE   = 3
EPOCHS = 3
ACCUMULATION_STEPS = 64
LR = 1e-4
WARMUP_RATIO = 0.1
CHECKPOINT_PATH = "checkpoint_latest.pth"

In [4]:
def save_checkpoint(path, model, optimizer, scheduler, epoch):
    torch.save({
        'epoch': epoch,
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict(),
        'scheduler_state': scheduler.state_dict()
    }, path)
    print(f"👉 Saved checkpoint: {path}")

In [5]:
def load_checkpoint(path, model, optimizer=None, scheduler=None, device='cpu'):
    ckpt = torch.load(path, map_location=device)
    model.load_state_dict(ckpt['model_state'])
    if optimizer: optimizer.load_state_dict(ckpt['optimizer_state'])
    if scheduler: scheduler.load_state_dict(ckpt['scheduler_state'])
    start_epoch = ckpt['epoch'] + 1
    print(f"✅ Loaded checkpoint from {path}, resuming at epoch {start_epoch}")
    return start_epoch

In [6]:
# 1) PositionalEncoding giữ nguyên như bạn đã định nghĩa

class PositionalEncoding(nn.Module):
    def __init__(self, d_model:int, max_len:int=5000, learned:bool=False):
        super().__init__()
        if learned:
            self.pe = nn.Parameter(torch.randn(1, max_len, d_model))
        else:
            pe = torch.zeros(max_len, d_model)
            pos = torch.arange(max_len).unsqueeze(1).float()
            div = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0)/d_model))
            pe[:,0::2] = torch.sin(pos * div)
            pe[:,1::2] = torch.cos(pos * div)
            self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        # x: (B, L, D)
        B,L,D = x.shape
        assert L <= self.pe.size(1)
        return x + self.pe[:, :L, :]

In [7]:
# 2) ScratchViTEncoder dynamic số patch

class ScratchViTEncoder(nn.Module):
    def __init__(self,
                 hidden_dim: int = 768,
                 img_size:    int = 448,
                 learned:     bool = True):
        super().__init__()
        # load config, tắt flash_attn + checkpointing
        config = AutoConfig.from_pretrained(
            "OpenGVLab/InternViT-300M-448px", trust_remote_code=True
        )
        config.use_flash_attn = False
        config.gradient_checkpointing = False

        # load model
        self.vit = AutoModel.from_pretrained(
            "OpenGVLab/InternViT-300M-448px",
            config=config,
            torch_dtype=torch.bfloat16,
            low_cpu_mem_usage=True,
            trust_remote_code=True
        ).to(device).eval()

        # CLIP processor
        self.processor = CLIPImageProcessor.from_pretrained(
            "OpenGVLab/InternViT-300M-448px"
        )

        # freeze except block 11
        for name, p in self.vit.named_parameters():
            p.requires_grad = ("blocks.11" in name)

        # project head
        self.proj = nn.Linear(config.hidden_size, hidden_dim).to(device)

        # compute num_patches dynamically
        self.patch_size = config.patch_size
        self.img_size   = img_size
        self.num_patches = (img_size // config.patch_size) ** 2 + 1

        # positional encoding (fixed or learned)
        if learned:
            self.pos_enc = nn.Parameter(torch.zeros(1, self.num_patches, hidden_dim))
            nn.init.normal_(self.pos_enc, 0, 0.02)
            self._learned = True
        else:
            pe = torch.zeros(self.num_patches, hidden_dim)
            pos = torch.arange(self.num_patches).unsqueeze(1).float()
            div = torch.exp(torch.arange(0, hidden_dim, 2).float() *
                            -(math.log(10000.0)/hidden_dim))
            pe[:,0::2] = torch.sin(pos * div)
            pe[:,1::2] = torch.cos(pos * div)
            self.register_buffer('pe', pe.unsqueeze(0))
            self._learned = False

    def forward(self, images):
        # images: list[PIL] or Tensor(B,3,448,448)
        if isinstance(images, list):
            batch = self.processor(images=images, return_tensors="pt").to(device)
            pixel_values = batch["pixel_values"]
        else:
            pixel_values = images.to(device)

        # checkpoint to save memory
        outputs = checkpoint(
            self.vit,
            pixel_values.to(torch.bfloat16),
            use_reentrant=False
        )
        feats = outputs.last_hidden_state               # (B, N, cfg.hidden_size)
        feats = self.proj(feats.to(torch.float32))      # (B, N, hidden_dim)

        # add pos
        if self._learned:
            return feats + self.pos_enc[:, :feats.size(1), :]
        else:
            return feats + self.pe[:, :feats.size(1), :]

In [8]:
# === 3) Text Encoder: PhoBERT (frozen) ===
class ScratchTextEncoder(nn.Module):
    def __init__(self,
                 model_name="vinai/phobert-base",
                 hidden_dim=768,
                 max_len=MAX_LEN,
                 freeze=True):
        super().__init__()
        self.text_model = AutoModel.from_pretrained(model_name).to(device)
        if freeze:
            for p in self.text_model.parameters():
                p.requires_grad = False
        # project only if needed
        if hidden_dim == self.text_model.config.hidden_size:
            self.proj = nn.Identity()
        else:
            self.proj = nn.Linear(self.text_model.config.hidden_size, hidden_dim).to(device)
        self.pos_enc = PositionalEncoding(hidden_dim, max_len, learned=True).to(device)
        assert max_len == MAX_LEN

    def forward(self,
                input_ids: torch.LongTensor,
                attention_mask: torch.LongTensor) -> torch.Tensor:
        B, L = input_ids.shape
        assert L <= MAX_LEN
        outs = self.text_model(
            input_ids=input_ids.to(device),
            attention_mask=attention_mask.to(device)
        )
        feats = self.proj(outs.last_hidden_state)  # (B, L, hidden_dim)
        return self.pos_enc(feats)                 # + pos

In [9]:
class FusionBlock(nn.Module):
    def __init__(self, hidden_dim: int = 768, num_heads: int = 8):
        super().__init__()
        self.self_attn  = MultiheadAttention(hidden_dim, num_heads, batch_first=True)
        self.norm_self  = LayerNorm(hidden_dim)
        self.cross_q2i  = MultiheadAttention(hidden_dim, num_heads, batch_first=True)
        self.norm_q     = LayerNorm(hidden_dim)
        self.ffn        = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 4),
            nn.ReLU(),
            nn.Linear(hidden_dim * 4, hidden_dim)
        )
        self.norm_ffn   = LayerNorm(hidden_dim)

    def forward(self,
                img_feats: torch.Tensor,   # (B, N_img, D)
                txt_feats: torch.Tensor,   # (B, N_txt, D)
                txt_mask: torch.Tensor = None  # (B, N_txt)
               ) -> tuple[torch.Tensor, torch.Tensor]:
        B1, N_img, D1 = img_feats.shape
        B2, N_txt, D2 = txt_feats.shape
        # Chỉ kiểm tra batch và dimension đồng nhất
        assert B1 == B2 and D1 == D2, \
            f"Batch or dim mismatch: img {img_feats.shape}, txt {txt_feats.shape}"

        # 1) Self‐attention trên [txt; img]
        combined = torch.cat([txt_feats, img_feats], dim=1)  # (B, N_txt+N_img, D)
        sa_out, _ = self.self_attn(combined, combined, combined)
        combined = self.norm_self(combined + sa_out)

        # 2) Tách lại thành text và image streams
        txt_out = combined[:, :N_txt, :]  # (B, N_txt, D)
        img_out = combined[:, N_txt:, :]  # (B, N_img, D)

        # 3) Cross‐attention: text queries image
        q2i_out, _ = self.cross_q2i(query=txt_out,
                                    key=img_out,
                                    value=img_out)
        txt_out = self.norm_q(txt_out + q2i_out)
        txt_out = self.norm_ffn(txt_out + self.ffn(txt_out))

        # 4) Tạo memory padding mask để decoder dùng
        mem_key_padding_mask = None
        if txt_mask is not None:
            # txt_mask: True cho tokens, False cho pad → chúng ta cần mask các pad
            txt_pad = ~txt_mask.bool()                # (B, N_txt)
            img_pad = torch.zeros(B1, N_img, dtype=torch.bool, device=txt_mask.device)
            mem_key_padding_mask = torch.cat([txt_pad, img_pad], dim=1)  # (B, N_txt+N_img)

        # 5) Nối lại làm memory
        memory = torch.cat([txt_out, img_out], dim=1)  # (B, N_txt+N_img, D)

        return memory, mem_key_padding_mask

In [10]:
class GenerativeDecoder(nn.Module):
    def __init__(self, vocab_size, hidden_dim=768, nhead=8, num_layers=4, max_len=MAX_LEN):
        super().__init__()
        layer = TransformerDecoderLayer(
            d_model=hidden_dim,
            nhead=nhead,
            dim_feedforward=hidden_dim*4,
            dropout=0.1,
            batch_first=True
        )
        self.decoder   = TransformerDecoder(layer, num_layers=num_layers)
        self.embed     = nn.Embedding(vocab_size, hidden_dim, padding_idx=0)
        self.pos_enc   = PositionalEncoding(hidden_dim, max_len, learned=True)
        self.out_proj  = nn.Linear(hidden_dim, vocab_size, bias=True)
        nn.init.normal_(self.embed.weight, mean=0, std=0.02)
        nn.init.normal_(self.out_proj.weight, mean=0, std=0.02)

    def forward(self,
                memory: torch.Tensor,
                target_ids: torch.LongTensor,
                memory_key_padding_mask: torch.Tensor = None
               ) -> torch.Tensor:
        """
        memory: (B, S, D)
        target_ids: (B, T)
        memory_key_padding_mask: (B, S) mask for memory (True=pad)
        """
        B, T = target_ids.shape
        # embed + positional
        tgt = self.embed(target_ids.to(device))        # (B, T, D)
        tgt = self.pos_enc(tgt)                        # (B, T, D)
        # causal mask
        tgt_mask = torch.triu(torch.ones((T, T), dtype=torch.bool, device=device), diagonal=1)
        # pad mask for tgt
        tgt_key_padding_mask = target_ids.eq(0).to(device)
        # run decoder
        dec_out = self.decoder(
            tgt=tgt,
            memory=memory,
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=tgt_key_padding_mask,
            memory_key_padding_mask=memory_key_padding_mask
        )
        return self.out_proj(dec_out)

    @torch.no_grad()
    def generate(self,
                 memory: torch.Tensor,
                 bos_id: int,
                 eos_id: int,
                 max_len: int = MAX_LEN,
                 beam_size: int = BEAM_SIZE,
                 use_topk: bool = False,
                 top_k: int = 100,
                 memory_key_padding_mask: torch.Tensor = None
                ) -> torch.LongTensor:
        B = memory.size(0)
        device = memory.device
    
        if use_topk:
            seq = torch.full((B, 1), bos_id, dtype=torch.long, device=device)
            non_special = torch.zeros(B, dtype=torch.long, device=device)
    
            for _ in range(max_len - 1):
                logits = self.forward(memory, seq, memory_key_padding_mask)
                probs  = F.softmax(logits[:, -1], dim=-1)
                probs  = torch.nan_to_num(probs, nan=0.0, posinf=0.0, neginf=0.0)
                probs  = torch.clamp(probs, min=0.0)
                sums   = probs.sum(dim=-1, keepdim=True)
                zero_mask = sums.squeeze(-1).eq(0)
                if zero_mask.any():
                    probs[zero_mask] = 1.0
                    sums = probs.sum(dim=-1, keepdim=True)
                probs = probs / sums
                if seq.size(1) < 3:
                    probs[:, eos_id] *= 0.1
                    probs = probs / probs.sum(dim=-1, keepdim=True)
    
                top_probs, top_ids = probs.topk(top_k, dim=-1)
                idx = torch.multinomial(top_probs, 1)
                next_tok = top_ids.gather(1, idx)  # (B,1)
    
                # ---- Squeeze mask to shape (B,) before adding  ----
                mask = ((next_tok != bos_id) &
                        (next_tok != eos_id) &
                        (next_tok != 0)).squeeze(1)   # now shape (B,)
                non_special += mask.long()
    
                seq = torch.cat([seq, next_tok], dim=1)
                if (seq[:, -1] == eos_id).all() and non_special.min() >= 2:
                    break
    
            # fallback check...
            decoded = tokenizer.batch_decode(seq, skip_special_tokens=True)
            if all(d.strip()=="" for d in decoded):
                use_topk = False
            else:
                return seq

        # Beam search
        beams = [(torch.full((B,1), bos_id, dtype=torch.long, device=device), 0.0)]
        completed = []
        for _ in range(max_len - 1):
            new_beams = []
            for seq, score in beams:
                if (seq[:, -1] == eos_id).all() and seq.size(1) > 2:
                    completed.append((seq, score))
                    continue
                logits = self.forward(memory, seq, memory_key_padding_mask)
                probs = F.softmax(logits[:, -1], dim=-1)
                if seq.size(1) < 3:
                    probs[:, eos_id] *= 0.1
                top_probs, top_ids = probs.topk(beam_size, dim=-1)
                for k in range(beam_size):
                    new_seq = torch.cat([seq, top_ids[:, k:k+1]], dim=1)
                    new_score = score + torch.log(top_probs[:, k]).mean().item()
                    new_beams.append((new_seq, new_score))
            beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]
            if len(completed) >= beam_size:
                break

        all_beams = completed + beams
        best_seq, _ = max(all_beams, key=lambda x: x[1])

        # Trim at EOS
        final_seqs = []
        for s in best_seq.tolist():
            if eos_id in s[1:]:
                idx = s.index(eos_id, 1)
                final_seqs.append(s[:idx])
            else:
                final_seqs.append(s)
        L_out = max(len(s) for s in final_seqs)
        out = torch.full((B, L_out), bos_id, dtype=torch.long, device=device)
        for i, s in enumerate(final_seqs):
            out[i, :len(s)] = torch.tensor(s, device=device)
        return out

In [11]:
# 6) Full VQA Model (giữ nguyên như bạn có, sẽ gọi đúng decoder.forward signature)
class VQAModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim=768):
        super().__init__()
        self.img_enc = ScratchViTEncoder(hidden_dim, learned=True)
        self.txt_enc = ScratchTextEncoder("vinai/phobert-base", hidden_dim, MAX_LEN, freeze=True)
        self.fuse    = FusionBlock(hidden_dim, num_heads=8)
        self.decoder = GenerativeDecoder(vocab_size,
                                         hidden_dim,
                                         nhead=8,
                                         num_layers=4,
                                         max_len=MAX_LEN)

    def forward(self, pv, qid, qmk, aid):
        img_feats = self.img_enc(pv)
        txt_feats = self.txt_enc(qid, qmk)
        memory, mem_pad = self.fuse(img_feats, txt_feats, qmk)
        # now forward accepts memory_key_padding_mask
        return self.decoder(memory, aid, memory_key_padding_mask=mem_pad)

    def generate_answer(self, pv, qid, qmk, bos_id, eos_id, max_len = MAX_LEN, beam_size = BEAM_SIZE, use_topk = False, top_k = 100):
        img_feats = self.img_enc(pv)
        txt_feats = self.txt_enc(qid, qmk)
        memory, mem_pad = self.fuse(img_feats, txt_feats, qmk)
        return self.decoder.generate(memory, bos_id, eos_id, max_len = MAX_LEN, beam_size = BEAM_SIZE, use_topk=use_topk,top_k=top_k,memory_key_padding_mask=mem_pad)

In [12]:
# === 7) Dataset & Preprocess ===
def preprocess_text(text):
    text = text.replace('_', ' ')
    tokens = word_tokenize(text)       # underthesea mới
    return " ".join(tokens) if isinstance(tokens, list) else tokens

class VQADataset(Dataset):
    def __init__(self, df, tokenizer, max_len=MAX_LEN, use_ocr=False):
        # 1) Loại bỏ các bản ghi có question hoặc answer sau strip() là rỗng
        mask = (df['question'].str.strip() != "") & (df['answers'].str.strip() != "")
        df = df[mask].reset_index(drop=True)

        # 2) Gom nhóm theo image_path
        self.df = df.groupby('path') \
                    .agg({'question': list, 'answers': list}) \
                    .reset_index()

        self.tokenizer = tokenizer
        self.max_len   = max_len
        self.use_ocr   = use_ocr

        # 3) CLIP processor để tạo pixel_values
        self.processor = CLIPImageProcessor.from_pretrained(
            "OpenGVLab/InternViT-300M-448px"
        )

        # 4) Nếu dùng OCR, khởi tạo reader và cache trước
        if use_ocr:
            self.ocr = easyocr.Reader(['vi'], gpu=torch.cuda.is_available())
            self.ocr_cache = {}
            for path in self.df['path'].unique():
                try:
                    res = self.ocr.readtext(path, detail=0)
                    self.ocr_cache[path] = " ".join(res) if res else ""
                except:
                    self.ocr_cache[path] = ""

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # A) Pixel values
        pv = self.processor(
            images=Image.open(row['path']).convert('RGB'),
            return_tensors="pt"
        )["pixel_values"].squeeze(0)

        # B) Chọn ngẫu nhiên 1 cặp Q&A
        i = random.randint(0, len(row['question']) - 1)
        ques = preprocess_text(row['question'][i])

        # C) Ghép OCR nếu cần
        if self.use_ocr:
            img_text = self.ocr_cache.get(row['path'], "")
            if img_text:
                ques = f"{ques} [IMG_TEXT] {img_text}"

        # D) Mã hóa question
        enc_q = self.tokenizer(
            ques,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )

        # E) Mã hóa answer (thêm BOS/EOS)
        ans = preprocess_text(row['answers'][i])
        ans = f"{self.tokenizer.bos_token} {ans} {self.tokenizer.eos_token}"
        enc_a = self.tokenizer(
            ans,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            'pixel_values': pv,
            'question_ids':  enc_q.input_ids.squeeze(0),
            'question_mask': enc_q.attention_mask.squeeze(0),
            'answer_ids':    enc_a.input_ids.squeeze(0)
        }

In [13]:
def read_json(path):
    with open(path, 'r') as f:
        data = json.load(f)
        images_list = data['annotations']
        return pd.DataFrame(images_list)

In [14]:
df_train = read_json("/kaggle/input/vitextvqa/ViTextVQA_json/ViTextVQA_train.json")
df_test = read_json("/kaggle/input/testdataset-cv/data_test_cv/My_ViTextVQA_test.json")
df_val = read_json("/kaggle/input/vitextvqa/ViTextVQA_json/ViTextVQA_dev.json")
# df_train = df_train.iloc[0:10]
df_train = pd.concat([df_train, df_val])
df_train

,id,image_id,question,answers
0,1,0,quán ăn này bán những món gì ?,"[mì quảng , bún bò huế]"
1,2,0,quán ăn này bán mì gì ?,[mì quảng]
2,3,0,quán ăn gì bán bún gì ?,[bún bò huế]
3,4,0,quán ăn này địa chỉ ở đâu ?,"[13a trần quốc thảo , phường 7 , quận 3]"
4,5,1,tiệm trong ảnh là tiệm gì ?,[sửa xe]
...,...,...,...,...
5150,50295,16741,bảng này do công ty tnhh kỹ thuật nào sản xuất ?,[pmit]
5151,50296,16741,công ty pmit là công ty gì ?,[tnhh kỹ thuật]
5152,50297,16741,email của công ty pmit là gì ?,[pmit . general@gmail . com]
5153,50298,16741,số điện thoại của công ty pmit là gì ?,[0226 532544]


In [15]:
def format_image(root_dir, df_1):
    df_1["path"] = df_1['image_id'].apply(lambda x: os.path.join(root_dir, str(x) + ".jpg"))
    df_1 = df_1.dropna(subset=['question','answers']).copy()
    df_1['question'] = df_1['question'].astype(str)
    df_1['answers'] = df_1['answers'].astype(str)
    df_1 = df_1.drop(["id"], axis = 1)
    df_1.reset_index(drop=True, inplace=True)
    return df_1

In [16]:
root_dir = '/kaggle/input/vitextvqa/ViTextVQA_images/st_images'
root_dir_test = "/kaggle/input/testdataset-cv/data_test_cv/image"
df_train = format_image(root_dir, df_train)
df_test = format_image(root_dir_test, df_test)
df_train

,image_id,question,answers,path
0,0,quán ăn này bán những món gì ?,"['mì quảng , bún bò huế']",/kaggle/input/vitextvqa/ViTextVQA_images/st_im...
1,0,quán ăn này bán mì gì ?,['mì quảng'],/kaggle/input/vitextvqa/ViTextVQA_images/st_im...
2,0,quán ăn gì bán bún gì ?,['bún bò huế'],/kaggle/input/vitextvqa/ViTextVQA_images/st_im...
3,0,quán ăn này địa chỉ ở đâu ?,"['13a trần quốc thảo , phường 7 , quận 3']",/kaggle/input/vitextvqa/ViTextVQA_images/st_im...
4,1,tiệm trong ảnh là tiệm gì ?,['sửa xe'],/kaggle/input/vitextvqa/ViTextVQA_images/st_im...
...,...,...,...,...
40309,16741,bảng này do công ty tnhh kỹ thuật nào sản xuất ?,['pmit'],/kaggle/input/vitextvqa/ViTextVQA_images/st_im...
40310,16741,công ty pmit là công ty gì ?,['tnhh kỹ thuật'],/kaggle/input/vitextvqa/ViTextVQA_images/st_im...
40311,16741,email của công ty pmit là gì ?,['pmit . general@gmail . com'],/kaggle/input/vitextvqa/ViTextVQA_images/st_im...
40312,16741,số điện thoại của công ty pmit là gì ?,['0226 532544'],/kaggle/input/vitextvqa/ViTextVQA_images/st_im...


In [17]:
print("Size train data:", len(df_train))
print("Sample train data:", df_train.head())
print("Size test data:", len(df_test))
print("Sample test data:", df_test.head())

Size train data: 40314
Sample train data:    image_id                        question  \
0         0  quán ăn này bán những món gì ?   
1         0         quán ăn này bán mì gì ?   
2         0         quán ăn gì bán bún gì ?   
3         0     quán ăn này địa chỉ ở đâu ?   
4         1     tiệm trong ảnh là tiệm gì ?   

                                      answers  \
0                   ['mì quảng , bún bò huế']   
1                                ['mì quảng']   
2                              ['bún bò huế']   
3  ['13a trần quốc thảo , phường 7 , quận 3']   
4                                  ['sửa xe']   

                                                path  
0  /kaggle/input/vitextvqa/ViTextVQA_images/st_im...  
1  /kaggle/input/vitextvqa/ViTextVQA_images/st_im...  
2  /kaggle/input/vitextvqa/ViTextVQA_images/st_im...  
3  /kaggle/input/vitextvqa/ViTextVQA_images/st_im...  
4  /kaggle/input/vitextvqa/ViTextVQA_images/st_im...  
Size test data: 100
Sample test data:   image_id  

In [18]:
bert_model = AutoModel.from_pretrained("vinai/phobert-base").to(device)

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [19]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
tokenizer.add_special_tokens({'bos_token': '<s>', 'eos_token': '</s>'})
print("BOS token:", tokenizer.bos_token, "EOS token:", tokenizer.eos_token)

train_ds = VQADataset(df_train, tokenizer, max_len=MAX_LEN, use_ocr=False)
val_ds = VQADataset(df_test, tokenizer, max_len=MAX_LEN, use_ocr=False)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=0, pin_memory=True)

model = VQAModel(vocab_size=len(tokenizer), hidden_dim=768).to(device)

bert_emb = model.txt_enc.text_model.embeddings.word_embeddings.weight
model.decoder.embed = nn.Embedding.from_pretrained(bert_emb, freeze=False, padding_idx=tokenizer.pad_token_id).to(device)
model.decoder.out_proj = nn.Linear(bert_emb.size(1), bert_emb.size(0)).to(device)
model.decoder.out_proj.weight = model.decoder.embed.weight
nn.init.zeros_(model.decoder.out_proj.bias)

optimizer = AdamW(model.parameters(), lr=LR)
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(WARMUP_RATIO * total_steps),
    num_training_steps=total_steps
)
scaler = torch.cuda.amp.GradScaler()
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

start_epoch = 0
if os.path.exists(CHECKPOINT_PATH):
    start_epoch = load_checkpoint(CHECKPOINT_PATH, model, optimizer, scheduler, device)

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

BOS token: <s> EOS token: </s>


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

...

modeling_intern_vit.py:   0%|          | 0.00/14.9k [00:00<?, ?B/s]

flash_attention.py:   0%|          | 0.00/3.46k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternViT-300M-448px:
- flash_attention.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternViT-300M-448px:
- modeling_intern_vit.py
- flash_attention.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


model.safetensors:   0%|          | 0.00/608M [00:00<?, ?B/s]

/tmp/ipykernel_20/1023068309.py:25: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [20]:
def evaluate_model(model, loader, tokenizer, epoch):
    model.eval()
    preds, refs = [], []
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            pv = batch['pixel_values'].to(device)
            qid = batch['question_ids'].to(device)
            qmk = batch['question_mask'].to(device)
            aid = batch['answer_ids'].to(device)
            
            gen_ids = model.generate_answer(pv, qid, qmk, tokenizer.bos_token_id, tokenizer.eos_token_id, MAX_LEN, use_topk=True)
            ptxt = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)
            print(f"gen_ids: {gen_ids.tolist()[:5]}")
            if all(p.strip() == "" for p in ptxt):
                print("Empty predictions detected, retrying with beam search")
                gen_ids = model.generate_answer(pv, qid, qmk, tokenizer.bos_token_id, tokenizer.eos_token_id, MAX_LEN, use_topk=False)
                ptxt = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)
            
            rtxt = tokenizer.batch_decode(aid, skip_special_tokens=True)
            if len(preds) < 5:
                print(f"Prediction: {ptxt[0]}")
                print(f"Reference: {rtxt[0]}")
            preds.extend(ptxt)
            refs.extend(rtxt)
    
    with open(f"predictions_epoch_{epoch+1}.txt", "w", encoding="utf-8") as f:
        for p, r in zip(preds[:10], refs[:10]):
            f.write(f"Prediction: {p}\nReference: {r}\n\n")
    
    bleu = corpus_bleu([[r.split()] for r in refs], [p.split() for p in preds])
    rouge1 = sum(scorer.score(r, p)['rouge1'].fmeasure for r, p in zip(refs, preds)) / len(preds)
    rougeL = sum(scorer.score(r, p)['rougeL'].fmeasure for r, p in zip(refs, preds)) / len(preds)
    print(f"Epoch {epoch+1} - Test Evaluation: BLEU: {bleu:.4f}, ROUGE-1: {rouge1:.4f}, ROUGE-L: {rougeL:.4f}")
    return bleu, rouge1, rougeL

In [21]:
torch.cuda.empty_cache()
model.train()
for epoch in range(start_epoch, EPOCHS):
    total_loss = 0.0
    optimizer.zero_grad()
    for i, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")):
        pv = batch['pixel_values'].to(device)
        qid = batch['question_ids'].to(device)
        qmk = batch['question_mask'].to(device)
        aid = batch['answer_ids'].to(device)

        with torch.amp.autocast('cuda'):
            logits = model(pv, qid, qmk, aid)
            loss = criterion(logits.view(-1, logits.size(-1)), aid.view(-1)) / ACCUMULATION_STEPS

        scaler.scale(loss).backward()
        total_loss += loss.item() * ACCUMULATION_STEPS

        if (i+1) % ACCUMULATION_STEPS == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()
            if torch.cuda.memory_reserved() / 1024**2 > 1024:
                torch.cuda.empty_cache()
            print(f"Step {i+1}: Loss: {loss.item() * ACCUMULATION_STEPS:.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{EPOCHS} — Loss: {avg_loss:.4f}")
    print(f"Allocated: {torch.cuda.memory_allocated()/1024**2:.2f} MiB, Reserved: {torch.cuda.memory_reserved()/1024**2:.2f} MiB")
    save_checkpoint(CHECKPOINT_PATH, model, optimizer, scheduler, epoch)

    bleu, r1, rL = evaluate_model(model, val_loader, tokenizer, epoch)
    print(f"Epoch {epoch+1} - Test Evaluation: BLEU: {bleu:.4f}, ROUGE-1: {r1:.4f}, ROUGE-L: {rL:.4f}")
    model.train()

Epoch 1/3:   0%|          | 0/839 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Epoch 1/3:   8%|▊         | 64/839 [03:06<41:10,  3.19s/it]

Step 64: Loss: 10.5140


Epoch 1/3:  15%|█▌        | 128/839 [06:30<38:32,  3.25s/it]

Step 128: Loss: 10.3468


Epoch 1/3:  23%|██▎       | 192/839 [09:54<34:24,  3.19s/it]

...

gen_ids: [[0, 72, 16, 87, 57, 20, 46, 91, 94, 67, 40, 36, 84, 55, 59], [0, 86, 22, 93, 11, 0, 6, 47, 70, 43, 44, 39, 32, 73, 78], [0, 47, 92, 30, 2, 76, 81, 42, 69, 62, 73, 92, 1, 23, 17], [0, 20, 83, 21, 28, 37, 71, 44, 80, 31, 49, 65, 0, 98, 40], [0, 12, 96, 47, 69, 29, 11, 93, 7, 12, 36, 42, 81, 16, 11]]
Prediction: lên một rằng đi ( ông do cả nhất ra nhiều cao biết rất
Reference: [ ' burger papa ' ]


Evaluating:  67%|██████▋   | 2/3 [00:17<00:08,  8.75s/it]

gen_ids: [[0, 54, 73, 32, 45, 96, 28, 76, 57, 74, 14, 36, 90, 91, 72], [0, 73, 32, 52, 73, 49, 26, 86, 26, 59, 86, 66, 23, 83, 76], [0, 12, 96, 28, 17, 38, 22, 34, 30, 20, 57, 46, 97, 81, 45], [0, 89, 76, 40, 68, 71, 11, 53, 33, 56, 56, 47, 62, 88, 74], [0, 23, 65, 54, 60, 66, 98, 67, 11, 63, 18, 11, 51, 39, 35]]


Evaluating: 100%|██████████| 3/3 [00:19<00:00,  6.40s/it]

gen_ids: [[0, 4, 6, 72, 23, 53, 72, 27, 91, 36, 9, 27, 71, 52, 59], [0, 87, 28, 36, 75, 30, 7, 15, 10, 84, 27, 89, 30, 99, 60], [0, 70, 56, 61, 13, 80, 66, 86, 2, 89, 35, 4, 54, 77, 54]]
Epoch 3 - Test Evaluation: BLEU: 0.0000, ROUGE-1: 0.1158, ROUGE-L: 0.0988
Epoch 3 - Test Evaluation: BLEU: 0.0000, ROUGE-1: 0.1158, ROUGE-L: 0.0988
